In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [2]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 0


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization

In [5]:
optimizer_init_file_name = "optimizer/optimizer_init.json"

In [6]:
ax_client_init = sdl.initialize_ax()
ax_client_init.save_to_json_file(optimizer_init_file_name)

[WARNING 07-28 16:10:15] ax.service.utils.instantiation: Objective thresholds were not set. They will be selected using a heuristic, but should be specified on the objective for best performance.


# Generate recommendations

In [7]:
time_start = time.time()

# generate recomms
ax_client_init = AxClient.load_from_json_file(optimizer_init_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=0)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

[INFO 07-28 16:10:15] ax.service.ax_client: Generated new trial 0 with parameters {'Drug': 48, 'SL_1': 59, 'SL_2': 50, 'SL_3': 31, 'LL_1': 96, 'LL_2': 8, 'LL_3': 9, 'P_1': 35, 'P_2': 85, 'P_3': 76, 'S_1': 89, 'S_2': 89, 'S_3': 80, 'Water': 85} using model Sobol.
[INFO 07-28 16:10:15] ax.service.ax_client: Generated new trial 1 with parameters {'Drug': 93, 'SL_1': 37, 'SL_2': 50, 'SL_3': 84, 'LL_1': 50, 'LL_2': 66, 'LL_3': 78, 'P_1': 53, 'P_2': 26, 'P_3': 20, 'S_1': 37, 'S_2': 5, 'S_3': 28, 'Water': 0} using model Sobol.
[INFO 07-28 16:10:15] ax.service.ax_client: Generated new trial 2 with parameters {'Drug': 51, 'SL_1': 77, 'SL_2': 12, 'SL_3': 24, 'LL_1': 19, 'LL_2': 95, 'LL_3': 27, 'P_1': 92, 'P_2': 2, 'P_3': 28, 'S_1': 16, 'S_2': 30, 'S_3': 60, 'Water': 52} using model Sobol.
[INFO 07-28 16:10:15] ax.service.ax_client: Generated new trial 3 with parameters {'Drug': 9, 'SL_1': 20, 'SL_2': 88, 'SL_3': 72, 'LL_1': 73, 'LL_2': 37, 'LL_3': 60, 'P_1': 21, 'P_2': 62, 'P_3': 75, 'S_1': 59, 

0 min


In [8]:
ax_client_design.get_trials_data_frame()

,trial_index,arm_name,trial_status,generation_node,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
0,0,0_0,RUNNING,GenerationStep_0,48,59,50,31,96,8,9,35,85,76,89,89,80,85
1,1,1_0,RUNNING,GenerationStep_0,93,37,50,84,50,66,78,53,26,20,37,5,28,0
2,2,2_0,RUNNING,GenerationStep_0,51,77,12,24,19,95,27,92,2,28,16,30,60,52
3,3,3_0,RUNNING,GenerationStep_0,9,20,88,72,73,37,60,21,62,75,59,64,11,38
4,4,4_0,RUNNING,GenerationStep_0,23,93,75,53,54,24,49,77,22,5,7,56,71,19
5,5,5_0,RUNNING,GenerationStep_0,69,10,25,0,1,56,64,10,63,97,66,38,19,97
6,6,6_0,RUNNING,GenerationStep_0,76,75,88,92,30,78,13,47,91,52,86,13,89,44
7,7,7_0,RUNNING,GenerationStep_0,34,29,12,43,78,46,100,66,50,45,38,81,40,66
8,8,8_0,RUNNING,GenerationStep_0,28,81,98,9,38,30,40,85,14,69,48,94,14,37
9,9,9_0,RUNNING,GenerationStep_0,83,15,2,61,92,100,73,1,73,35,77,11,63,73


In [9]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_1,S_2,S_3,Water,Solu,Solu_STD,Size,Size_STD,PDI,PDI_STD
0,0,0.097,0.119,0.101,0.062,0.193,0.016,0.018,0.070,0.171,...,0.259,0.259,0.233,0.248,None,None,None,None,None,None
1,1,0.167,0.066,0.090,0.151,0.090,0.118,0.140,0.095,0.047,...,0.529,0.071,0.400,0.000,None,None,None,None,None,None
2,2,0.119,0.180,0.028,0.056,0.044,0.222,0.063,0.215,0.005,...,0.101,0.190,0.380,0.329,None,None,None,None,None,None
3,3,0.017,0.039,0.170,0.139,0.141,0.072,0.116,0.041,0.120,...,0.343,0.372,0.064,0.221,None,None,None,None,None,None
4,4,0.048,0.196,0.158,0.112,0.114,0.051,0.103,0.162,0.046,...,0.046,0.366,0.464,0.124,None,None,None,None,None,None
5,5,0.175,0.025,0.063,0.000,0.003,0.142,0.162,0.025,0.159,...,0.300,0.173,0.086,0.441,None,None,None,None,None,None
6,6,0.118,0.117,0.137,0.143,0.047,0.121,0.020,0.073,0.142,...,0.371,0.056,0.384,0.190,None,None,None,None,None,None
7,7,0.068,0.058,0.024,0.085,0.155,0.091,0.199,0.131,0.099,...,0.169,0.360,0.178,0.293,None,None,None,None,None,None
8,8,0.057,0.165,0.199,0.018,0.077,0.061,0.081,0.173,0.028,...,0.249,0.487,0.073,0.192,None,None,None,None,None,None
9,9,0.155,0.028,0.004,0.114,0.172,0.187,0.136,0.002,0.136,...,0.344,0.049,0.281,0.326,None,None,None,None,None,None


In [10]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.000000,16.0000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,7.500000,50.375000,50.000000,50.000000,49.7500,50.125000,50.937500,50.062500,49.875000,50.125000,49.875000,50.062500,50.062500,50.000000,50.000000
std,4.760952,30.236567,29.870833,30.351277,30.2732,30.097342,30.440037,30.612566,29.926298,30.340018,29.828119,29.945436,29.641117,30.035535,30.031095
min,0.000000,3.000000,5.000000,2.000000,0.0000,1.000000,1.000000,1.000000,1.000000,2.000000,5.000000,5.000000,5.000000,3.000000,0.000000
25%,3.750000,26.750000,26.750000,24.250000,27.7500,27.250000,28.500000,26.000000,25.500000,25.000000,26.000000,26.000000,27.500000,25.750000,29.250000
50%,7.500000,49.500000,48.500000,50.000000,49.0000,52.000000,51.000000,49.500000,50.000000,51.000000,48.500000,52.000000,50.500000,50.500000,48.000000
75%,11.250000,74.500000,75.500000,75.500000,74.0000,74.250000,72.750000,74.250000,73.250000,74.000000,75.250000,71.750000,72.000000,73.250000,74.500000
max,15.000000,100.000000,98.000000,98.000000,96.0000,96.000000,100.000000,100.000000,95.000000,100.000000,97.000000,99.000000,94.000000,98.000000,97.000000


In [11]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.00000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.0000,16.000000
mean,7.500000,34.518750,34.671875,35.371875,34.212500,34.015625,35.30625,35.765625,34.475000,36.596875,35.000000,257.187500,243.500000,254.250000,245.1875,0.065625
std,4.760952,18.970678,21.197586,21.602106,21.242242,19.076860,20.39307,22.151883,20.358651,23.010371,21.755252,141.617898,126.269553,147.276384,151.4557,0.318705
min,0.000000,2.800000,5.950000,1.400000,0.000000,1.050000,0.70000,1.050000,0.700000,1.750000,3.850000,19.000000,49.000000,23.000000,0.0000,-0.350000
25%,3.750000,19.162500,18.637500,20.912500,19.075000,16.187500,21.26250,17.850000,23.012500,16.362500,20.300000,154.250000,168.250000,151.250000,161.7500,-0.087500
50%,7.500000,37.975000,32.550000,33.425000,30.975000,36.050000,34.65000,38.325000,29.750000,38.325000,33.250000,260.500000,253.000000,245.000000,234.5000,0.000000
75%,11.250000,48.737500,51.975000,49.787500,49.000000,45.412500,46.28750,53.287500,49.175000,56.700000,49.437500,350.750000,354.750000,381.000000,326.7500,0.350000
max,15.000000,61.250000,68.600000,71.050000,76.650000,67.550000,77.70000,69.650000,75.250000,73.500000,86.100000,529.000000,487.000000,473.000000,608.0000,0.700000


In [12]:
print("For 3 mm aspiration height, dead volume of 20 mL vials is around 2.5 mL") 

round(transfer[['Drug', 'SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000, 2)

For 3 mm aspiration height, dead volume of 20 mL vials is around 2.5 mL


Drug       0.55
SL_1       0.55
SL_2       0.57
SL_3       0.55
LL_1       0.54
LL_2       0.56
LL_3       0.57
P_1        0.55
P_2        0.59
P_3        0.56
S_1        4.12
S_2        3.90
S_3        4.07
Water      3.92
Solvent    0.00
dtype: float64

In [13]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"
unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"
unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"
transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [14]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [15]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [16]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [17]:
updated_script

('protocol/iteration_0_OT_2_protocol_1.py',
 'protocol/iteration_0_OT_2_protocol_2.py')

# Perform experiment

# Results

In [18]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,0,0.097,0.119,0.101,0.062,0.193,0.016,0.018,0.070,0.171,0.153,0.259,0.259,0.233,0.248,12,0
1,1,0.167,0.066,0.090,0.151,0.090,0.118,0.140,0.095,0.047,0.036,0.529,0.071,0.400,0.000,12,0
2,2,0.119,0.180,0.028,0.056,0.044,0.222,0.063,0.215,0.005,0.066,0.101,0.190,0.380,0.329,12,0
3,3,0.017,0.039,0.170,0.139,0.141,0.072,0.116,0.041,0.120,0.145,0.343,0.372,0.064,0.221,12,0
4,4,0.048,0.196,0.158,0.112,0.114,0.051,0.103,0.162,0.046,0.011,0.046,0.366,0.464,0.124,12,0
5,5,0.175,0.025,0.063,0.000,0.003,0.142,0.162,0.025,0.159,0.246,0.300,0.173,0.086,0.441,11,0
6,6,0.118,0.117,0.137,0.143,0.047,0.121,0.020,0.073,0.142,0.081,0.371,0.056,0.384,0.190,12,0
7,7,0.068,0.058,0.024,0.085,0.155,0.091,0.199,0.131,0.099,0.089,0.169,0.360,0.178,0.293,12,0
8,8,0.057,0.165,0.199,0.018,0.077,0.061,0.081,0.173,0.028,0.140,0.249,0.487,0.073,0.192,12,0
9,9,0.155,0.028,0.004,0.114,0.172,0.187,0.136,0.002,0.136,0.065,0.344,0.049,0.281,0.326,12,0


In [19]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Caution,A1,154.725160,0.570796
1,Caution,A2,175.865028,0.570796
2,Caution,A3,164.578264,0.570796
3,Caution,A4,350.394651,0.570796
4,Caution,A5,345.142203,0.570796
5,Good,A6,320.922553,0.558092
6,Good,B1,331.010148,0.465873
7,Good,B2,306.276815,0.311858
8,Good,B3,312.501844,0.330593
9,Good,B4,244.465881,0.334494


In [20]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,Formulation 3,316.596269,12.864979,0.369441,0.084036,1
3,Formulation 4,252.343655,11.140854,0.359479,0.035334,1
4,Formulation 5,323.625351,2.417202,0.432044,0.031808,1
5,Formulation 6,225.073171,5.758277,0.299149,0.038186,1
6,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,Formulation 8,236.222805,6.701295,0.300872,0.048132,1
8,Formulation 9,266.542761,3.875382,0.387059,0.012545,1
9,Formulation 10,294.384922,8.616639,0.388992,0.050478,1


In [21]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,0,0.097,0.119,0.101,0.062,0.193,0.016,0.018,0.070,0.171,...,0.233,0.248,12,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,1,0.167,0.066,0.090,0.151,0.090,0.118,0.140,0.095,0.047,...,0.400,0.000,12,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,2,0.119,0.180,0.028,0.056,0.044,0.222,0.063,0.215,0.005,...,0.380,0.329,12,0,Formulation 3,316.596269,12.864979,0.369441,0.084036,1
3,3,0.017,0.039,0.170,0.139,0.141,0.072,0.116,0.041,0.120,...,0.064,0.221,12,0,Formulation 4,252.343655,11.140854,0.359479,0.035334,1
4,4,0.048,0.196,0.158,0.112,0.114,0.051,0.103,0.162,0.046,...,0.464,0.124,12,0,Formulation 5,323.625351,2.417202,0.432044,0.031808,1
5,5,0.175,0.025,0.063,0.000,0.003,0.142,0.162,0.025,0.159,...,0.086,0.441,11,0,Formulation 6,225.073171,5.758277,0.299149,0.038186,1
6,6,0.118,0.117,0.137,0.143,0.047,0.121,0.020,0.073,0.142,...,0.384,0.190,12,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,7,0.068,0.058,0.024,0.085,0.155,0.091,0.199,0.131,0.099,...,0.178,0.293,12,0,Formulation 8,236.222805,6.701295,0.300872,0.048132,1
8,8,0.057,0.165,0.199,0.018,0.077,0.061,0.081,0.173,0.028,...,0.073,0.192,12,0,Formulation 9,266.542761,3.875382,0.387059,0.012545,1
9,9,0.155,0.028,0.004,0.114,0.172,0.187,0.136,0.002,0.136,...,0.281,0.326,12,0,Formulation 10,294.384922,8.616639,0.388992,0.050478,1


In [22]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,0,0.097,0.119,0.101,0.062,0.193,0.016,0.018,0.070,0.171,...,12,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
1,1,0.167,0.066,0.090,0.151,0.090,0.118,0.140,0.095,0.047,...,12,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
2,2,0.119,0.180,0.028,0.056,0.044,0.222,0.063,0.215,0.005,...,12,0,Formulation 3,316.596269,12.864979,0.369441,0.084036,1,238.0,0
3,3,0.017,0.039,0.170,0.139,0.141,0.072,0.116,0.041,0.120,...,12,0,Formulation 4,252.343655,11.140854,0.359479,0.035334,1,34.0,0
4,4,0.048,0.196,0.158,0.112,0.114,0.051,0.103,0.162,0.046,...,12,0,Formulation 5,323.625351,2.417202,0.432044,0.031808,1,96.0,0
5,5,0.175,0.025,0.063,0.000,0.003,0.142,0.162,0.025,0.159,...,11,0,Formulation 6,225.073171,5.758277,0.299149,0.038186,1,350.0,0
6,6,0.118,0.117,0.137,0.143,0.047,0.121,0.020,0.073,0.142,...,12,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
7,7,0.068,0.058,0.024,0.085,0.155,0.091,0.199,0.131,0.099,...,12,0,Formulation 8,236.222805,6.701295,0.300872,0.048132,1,136.0,0
8,8,0.057,0.165,0.199,0.018,0.077,0.061,0.081,0.173,0.028,...,12,0,Formulation 9,266.542761,3.875382,0.387059,0.012545,1,114.0,0
9,9,0.155,0.028,0.004,0.114,0.172,0.187,0.136,0.002,0.136,...,12,0,Formulation 10,294.384922,8.616639,0.388992,0.050478,1,310.0,0


In [23]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,1.000000
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,1.000000
2,1,0.316596,0.012865,0.119,0.0,0.369441,0.084036,1.000000
3,1,0.252344,0.011141,0.017,0.0,0.359479,0.035334,1.000000
4,1,0.323625,0.002417,0.048,0.0,0.432044,0.031808,1.000000
5,1,0.225073,0.005758,0.175,0.0,0.299149,0.038186,0.916667
6,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,1.000000
7,1,0.236223,0.006701,0.068,0.0,0.300872,0.048132,1.000000
8,1,0.266543,0.003875,0.057,0.0,0.387059,0.012545,1.000000
9,1,0.294385,0.008617,0.155,0.0,0.388992,0.050478,1.000000


In [24]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [25]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [26]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,0,0.097,0.119,0.101,0.062,0.193,0.016,0.018,0.070,0.171,...,1.0,0,Formulation 1,1.000000,0.000000,1.000000,0.000000,0,0.000,0
1,1,0.167,0.066,0.090,0.151,0.090,0.118,0.140,0.095,0.047,...,1.0,0,Formulation 2,1.000000,0.000000,1.000000,0.000000,0,0.000,0
2,2,0.119,0.180,0.028,0.056,0.044,0.222,0.063,0.215,0.005,...,1.0,0,Formulation 3,0.316596,0.012865,0.369441,0.084036,1,0.119,0
3,3,0.017,0.039,0.170,0.139,0.141,0.072,0.116,0.041,0.120,...,1.0,0,Formulation 4,0.252344,0.011141,0.359479,0.035334,1,0.017,0
4,4,0.048,0.196,0.158,0.112,0.114,0.051,0.103,0.162,0.046,...,1.0,0,Formulation 5,0.323625,0.002417,0.432044,0.031808,1,0.048,0


In [27]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [28]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [29]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 07-28 16:10:15] ax.service.ax_client: Completed trial 0 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 07-28 16:10:15] ax.service.ax_client: Completed trial 1 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 07-28 16:10:15] ax.service.ax_client: Completed trial 2 with data: {'Solu': (0.119, 0), 'Size': (0.316596, 0.012865), 'PDI': (0.369441, 0.084036)}.
[INFO 07-28 16:10:15] ax.service.ax_client: Completed trial 3 with data: {'Solu': (0.017, 0), 'Size': (0.252344, 0.011141), 'PDI': (0.359479, 0.035334)}.
[INFO 07-28 16:10:15] ax.service.ax_client: Completed trial 4 with data: {'Solu': (0.048, 0), 'Size': (0.323625, 0.002417), 'PDI': (0.432044, 0.031808)}.
[INFO 07-28 16:10:15] ax.service.ax_client: Completed trial 5 with data: {'Solu': (0.175, 0), 'Size': (0.225073, 0.005758), 'PDI': (0.299149, 0.038186)}.
[INFO 07-28 16:10:15] ax.service.ax_client: Completed trial 6 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (